In [ ]:
################################### FP Tree method ###################################
## FP tree concept :
http://www.cnblogs.com/ybjourney/p/4851540.html
## scikit-learn :
http://hhtucode.blogspot.tw/2013/07/ml-python-scikit-learn-revision-1.html
## FP tree 原始碼來源 :
http://blog.csdn.net/bone_ace/article/details/46747669

In [1]:
#### step-1: import FP Tree package py file
#coding=utf-8

from tree_builder import Tree_builder
import tree_building
from tree_miner import Tree_miner
from collections import Counter


In [ ]:
[u'\u674e\u5728\u9394', u'\u4e09\u661f\u96c6\u5718', 2]

In [2]:
#### step-2: 自定函數

## fptree : 頻繁項目集生成
def fptree(routines, min_sup, headerTable, freq_items):
    # 建造FP_Tree
    treeBuilder = Tree_builder(routines=routines, min_sup=min_sup, headerTable=headerTable)
    
    # 生成頻繁項目集 
    treeMiner = Tree_miner(Tree=treeBuilder.tree, min_sup=min_sup, headerTable=headerTable, freq_items = freq_items)
    freqitems_set = treeMiner.tree_mine
    return freqitems_set

## fp_association : 產出強階層關聯規則
# freqitems_set:頻繁項目集 / keyword:關鍵字 / rank:取得關聯排序順位 / confcut:信賴度門檻
def fp_association(freqitems_set, keyword, rank, confcut) : 

    support_keyword = 0    # 查詢關鍵字的出現頻次 p(A)
    all_freq = 0    # 所有頻繁項目集出現頻次
    items = []
    for ele in freqitems_set :
        all_freq += float(ele[-1])
        if keyword in ele :
            support_keyword += float(ele[-1])
            items += [ word for word in ele[0:-1] if word != keyword ]
            items = list(set(items))
            
    conf = {}

    
    for item in items :
        support_union = 0    # 公同出現次數 p(A&B)
        support_other = 0    # p(B)
        confidence = 0   
        lift = 0
        for ele in freqitems_set :
            if item in ele :
                support_other += float(ele[-1])        
                if keyword in ele :
                    support_union += float(ele[-1])
        confidence = support_union/support_keyword    # p(B|A)
        lift = (support_union/support_keyword)/(support_other)*all_freq    # p(B|A)/p(B) > 1 表示正相關
        if lift > 1 and confidence > confcut :
                conf[item] = confidence
                
    top = dict(Counter(conf).most_common(rank))
    return top

In [1]:
#### step-3: 讀取所有文章內容 Tfidf 篩選後的關鍵字 --> 建議之後擴增為10個
import time
import pymongo
from pymongo import MongoClient

client = MongoClient('192.168.142.99', 27017)
dbname = ['news','blog','forum','social_media']
collectionname = ['Applenews_post','Ettoday_post','Setn_post','Pixnet_post','Ck101_post','Dcard_post','Eyny_post',\
                  'Mobile01_post','Ptt_post','FB_post']

routines = []
for dbn in dbname:
    for coln in collectionname:
    
        db = client[dbn]
        collection = db[coln]

        cursor = collection.find()
        cursor.count()
        ## 進入每篇 post 進行分析
        for document in cursor:
            routines.append(map(lambda ele : ele.encode('utf-8') , document["content_keywords"].split('|')))

print (len(routines))

19946


In [9]:
for ele in routines[4]:
    print ele

拆屋
反攻大陸
跳票
委員會
要說


In [64]:
#### step-4: 生成頻繁項目集

# 記錄開始時間 Record start time : 約 0.5分鐘
tStart = time.time()

min_sup = 2    #最小支持度计数
headerTable = {}    #头结点表，用来存放各个项的索引
freq_items = []
qq = fptree(routines, min_sup, headerTable, freq_items)

# 記錄結束時間 Record stop time
tStop = time.time()
print(tStop - tStart)

49.6500000954


In [70]:
f = open("FPtree_model" , 'w')
for ele in qq:
    f.write("{0}\n".format(ele))
f.close()

In [77]:
#### step-5: 運算查詢與 "關鍵字" 相關信賴度最高的 top10 字詞 
keyword = '台灣大'
rank = 10
confcut = 0.1 
top = fp_association(qq, keyword, rank, confcut)
for ele in top :
    print ele , top[ele]
print ('done')

## Top 10 向下一階層的關聯字詞 --> 排除上一層已出現的關鍵詞
for word in top.keys() :
    second_dict = fp_association(qq, word ,10, 0)
    third_dict = {k: v for k, v in second_dict.items() if k not in top.keys()}
    for ele in third_dict :
        print ele , third_dict[ele]
        print '='*100
    

#### --> 未來考量如何增加屬性詞的過濾網 

中華電 0.21554770318
鴻海 0.233215547703
持股 0.148409893993
強強 0.16961130742
done
蔡力行 0.0992907801418
軟體 0.0709219858156
方案 0.0756501182033
資本支出 0.108747044917
紅眼 0.10401891253
4g 0.12293144208
台灣大 0.144208037825
股權 0.113793103448
買家 0.113793103448
飛船 0.110344827586
國碁 0.110344827586
mhz 0.11724137931
台灣大 0.227586206897
應戰 0.110344827586
台灣之星 0.229007633588
買家 0.160305343511
增資 0.0458015267176
認購 0.0458015267176
頂新 0.160305343511
台灣大 0.320610687023
台灣大 0.533333333333


In [85]:
second_dict.items()

[('\xe4\xb8\xad\xe8\x8f\xaf\xe9\x9b\xbb', 0.5333333333333333),
 ('\xe9\xb4\xbb\xe6\xb5\xb7', 0.5333333333333333),
 ('\xe6\x8c\x81\xe8\x82\xa1', 0.5333333333333333),
 ('\xe5\x8f\xb0\xe7\x81\xa3\xe5\xa4\xa7', 0.5333333333333333)]

In [26]:
#### 驗證強關聯規則內容
tt = [ freq for freq in qq if '拋售' in freq ]
tt = sorted(tt, key = lambda x : x[-1] ,reverse=True)
tt[:3]

C:\Users\QQ\Anaconda2\envs\py27\lib\site-packages\ipykernel\__main__.py:2: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  from ipykernel import kernelapp as app


[]

In [9]:
keyword = 'sony'
rank = 10
top = fp_association(qq, keyword, rank, 0)
print (top)

{'z3': 0.16666666666666666, '平井一夫': 0.06060606060606061, '強在': 0.18181818181818182, '糾正': 0.12121212121212122, 'ui': 0.2727272727272727, '三星': 0.09090909090909091, 'tablet': 0.06060606060606061, '有錯': 0.12121212121212122, 'z5': 0.09090909090909091, 'm4': 0.06060606060606061}


In [ ]:
###################### 以下是開發測試過程 
#coding=utf-8
from tree_builder import Tree_builder
# import tree_builder
import tree_building
from tree_miner import Tree_miner

routines = [    
           ['Cola','Egg','Ham'],
           ['Cola','Diaper','Beer'],
           ['Cola','Beer','Diaper','Ham'],
           ['Diaper','Beer']
        ]                                  #事务数据集
min_sup = 2                             #最小支持度计数
headerTable = {}        #头结点表，用来存放各个项的索引
freq_items = []

treeBuilder = Tree_builder(routines=routines, min_sup=min_sup, headerTable=headerTable)    #建造FP_Tree
treeMiner = Tree_miner(Tree=treeBuilder.tree, min_sup=min_sup, headerTable=headerTable, freq_items = freq_items)

a = treeBuilder.getItems(routines)
b = treeBuilder.getSortedItems(a)
c = treeBuilder.initItemsTable(headerTable)
# print (a)
# print (b)
# print (c)
# for ele in treeBuilder.tree :
#     print (type(ele))

tt = treeMiner.tree_mine
print (tt)

In [ ]:
#### FP Tree support & confidence
keyword = 'Cola'
freqitems = tt
# print (freqitems)

support_keyword = 0
all_freq = 0
items = []
for ele in freqitems :
    all_freq += float(ele[-1])
    if keyword in ele :
        support_keyword += float(ele[-1])
        items += [ word for word in ele[0:-1] if word != keyword ]
        items = list(set(items))

# print (support_keyword) 
# print (all_freq)
# print (items)

conf = {}
for item in items :
    support_union = 0
    support_other = 0
    confidence = 0
    lift = 0
    for ele in freqitems :
        if item in ele :
            support_other += float(ele[-1])        
            if keyword in ele :
                support_union += float(ele[-1])
                confidence = support_union/support_keyword
                lift = (support_union/support_keyword)/(support_other)*all_freq
    if lift > 1 :
            conf[item] = confidence 
#             print (conf)
#     print ( item , support_union, support_other, confidence , lift)
# print (conf)

from collections import Counter
dict(Counter(conf).most_common(1))  